Dal database AdventureWorks estraiamo la tabella dimproduct • Sulla colonna DealerPrice, utilizzando il metodo .round(), arrotondiamo i valori alle due cifre decimali, e poi al valore intero più vicino • Utilizzando il metodo .clip(), facciamo in modo che i valori siano compresi tra un minimo di 0 e un massimo di 1000

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#importo le librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!pip install pymysql python-dotenv

import os
from dotenv import load_dotenv
import sqlalchemy

# Carico il file .env dalla directory
dotenv_path = "/content/drive/MyDrive/EPICODE/M4_PYTHON/credenzialiDB.env"
load_dotenv(dotenv_path=dotenv_path, override=True)

# Verifica caricamento
print("✅ username:", os.getenv("username"))

# Estrazione variabili
username = os.getenv("username")
password = os.getenv("password")
host = os.getenv("host")
dbname = os.getenv("dbname")

# Verifica variabili mancanti
missing = [k for k, v in {"username": username, "password": password, "host": host, "dbname": dbname}.items() if v is None]
if missing:
    raise EnvironmentError(f"❌ Variabili d'ambiente mancanti: {', '.join(missing)}")

# 🔗 Costruzione della stringa di connessione
connection_string = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"
print("🔗 Connection string costruita correttamente.")

# ⚙️ Creazione dell'engine SQLAlchemy
engine = sqlalchemy.create_engine(connection_string)

# 🔍 Test connessione
try:
    with engine.connect() as conn:
        print("✅ Connessione riuscita al database!")
except Exception as e:
    print("❌ Errore di connessione:", e)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00
✅ username: studente_dapt
🔗 Connection string costruita correttamente.
✅ Connessione riuscita al database!


In [4]:
#importo il df
query_dimproduct = "SELECT * FROM dimproduct"
df_dimproduct = pd.read_sql(query_dimproduct, engine)
df_dimproduct.info()
df_dimproduct.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ProductKey             606 non-null    int64         
 1   ProductAlternateKey    606 non-null    object        
 2   ProductSubcategoryKey  397 non-null    float64       
 3   WeightUnitMeasureCode  282 non-null    object        
 4   SizeUnitMeasureCode    253 non-null    object        
 5   EnglishProductName     606 non-null    object        
 6   SpanishProductName     606 non-null    object        
 7   FrenchProductName      606 non-null    object        
 8   StandardCost           397 non-null    float64       
 9   FinishedGoodsFlag      606 non-null    int64         
 10  Color                  606 non-null    object        
 11  SafetyStockLevel       606 non-null    int64         
 12  ReorderPoint           606 non-null    int64         
 13  ListP

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,...,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
0,1,AR-5381,NaN,None,None,Adjustable Race,,,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
1,2,BA-8327,NaN,None,None,Bearing Ball,,,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
2,3,BE-2349,NaN,None,None,BB Ball Bearing,,,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,,,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
4,5,BL-2036,NaN,None,None,Blade,,,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current


In [5]:
#Sulla colonna DealerPrice, utilizzando il metodo .round(), arrotondiamo i valori alle due cifre decimali, e poi al valore intero più vicino
#Utilizzando il metodo .clip(), facciamo in modo che i valori siano compresi tra un minimo di 0 e un massimo di 1000

df_DP_round_clip = df_dimproduct['DealerPrice'].round(2).clip(0, 1000)
df_DP_round_clip.value_counts()

,count
DealerPrice,
1000.00,50
202.33,12
469.79,12
419.46,12
183.94,12
...,...
27.65,1
54.89,1
12.14,1


Creiamo un DataFrame sintetico, che contiene i guadagni mensili di diverse annate, con il seguente codice: years = 5 guadagni = pd.DataFrame({"Mese": list("GFMAMGLASOND"*years), "Anno": np.repeat(list(range(years)), 12), "Valore": np.random.randint(800, 5000, 12*years)}) • Calcola la somma cumulativa dei guadagni utilizzando il metodo .cumsum() • Come sopra, ma raggruppato per ogni anno usando prima un .groupby()

In [6]:
#df sintetico
years = 5
guadagni = pd.DataFrame({"Mese": list("GFMAMGLASOND"*years), "Anno": np.repeat(list(range(years)), 12), "Valore": np.random.randint(800, 5000, 12*years)})

#creo colonna con soma cumulativa
guadagni["guadagni_cumsum"] = guadagni['Valore'].cumsum()
#creo variabile contenente la somma dei guadagni cumulativi per anno
guadagni_cum_anno = guadagni.groupby("Anno")['guadagni_cumsum'].sum().rename("guadagni_cumsum_mean")
print(guadagni)
print()
print(guadagni_cum_anno)

   Mese  Anno  Valore  guadagni_cumsum
0     G     0    2741             2741
1     F     0    3688             6429
2     M     0     866             7295
3     A     0    2271             9566
4     M     0    1925            11491
5     G     0     994            12485
6     L     0    4645            17130
7     A     0    4402            21532
8     S     0    3611            25143
9     O     0    2695            27838
10    N     0    4110            31948
11    D     0    1637            33585
12    G     1    4965            38550
13    F     1    2263            40813
14    M     1    4101            44914
15    A     1    2124            47038
16    M     1    4220            51258
17    G     1    2549            53807
18    L     1    4841            58648
19    A     1    4574            63222
20    S     1    1936            65158
21    O     1    3092            68250
22    N     1     863            69113
23    D     1    1908            71021
24    G     2    3839    

Dal database AdventureWorks estraiamo la tabella dimcustomer • Trasformiamo i nomi dei clienti in modo che abbiano solo lettere minuscole, e i cognomi in modo che abbiano solo lettere maiuscole • Sulla colonna EmailAddress, utilizzando il metodo .str.split(), estraiamo nome utente e dominio • Sulla colonna Phone, estraiamo ogni parte del numero (ad es. da "1 (11) 500 555-0162" a ["1", "(11)", "500", "555-0162"]) • Utilizzando il metodo .str.contains(), estraiamo tutti gli indirizzi e-mail che contengono il numero "21"

In [7]:
#importo il df
query_dimcustomer = "SELECT * FROM dimcustomer"
df_dimcustomer = pd.read_sql(query_dimcustomer, engine)
df_dimcustomer.info()
df_dimcustomer.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerKey           18484 non-null  int64  
 1   GeographyKey          18484 non-null  int64  
 2   CustomerAlternateKey  18484 non-null  object 
 3   Title                 101 non-null    object 
 4   FirstName             18484 non-null  object 
 5   MiddleName            10654 non-null  object 
 6   LastName              18484 non-null  object 
 7   NameStyle             18484 non-null  int64  
 8   BirthDate             18484 non-null  object 
 9   MaritalStatus         18484 non-null  object 
 10  Suffix                3 non-null      object 
 11  Gender                18484 non-null  object 
 12  EmailAddress          18484 non-null  object 
 13  YearlyIncome          18484 non-null  float64
 14  TotalChildren         18484 non-null  int64  
 15  NumberChildrenAtHom

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,11000,26,AW00011000,None,Jon,V,Yang,0,1971-10-06,M,...,Professional,Profesional,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles
1,11001,37,AW00011001,None,Eugene,L,Huang,0,1976-05-10,S,...,Professional,Profesional,Cadre,0,1,2243 W St.,None,1 (11) 500 555-0110,2011-01-15,0-1 Miles
2,11002,31,AW00011002,None,Ruben,None,Torres,0,1971-02-09,M,...,Professional,Profesional,Cadre,1,1,5844 Linden Land,None,1 (11) 500 555-0184,2011-01-07,2-5 Miles
3,11003,11,AW00011003,None,Christy,None,Zhu,0,1973-08-14,S,...,Professional,Profesional,Cadre,0,1,1825 Village Pl.,None,1 (11) 500 555-0162,2010-12-29,5-10 Miles
4,11004,19,AW00011004,None,Elizabeth,None,Johnson,0,1979-08-05,S,...,Professional,Profesional,Cadre,1,4,7553 Harness Circle,None,1 (11) 500 555-0131,2011-01-23,1-2 Miles


In [8]:
#trasformo i nomi in minuscolo e i cognomi in maiuscolo e li inserisco in due nuove colonne
df_dimcustomer["FirstName"] = df_dimcustomer["FirstName"].str.lower()
df_dimcustomer["LastName"] = df_dimcustomer['LastName'].str.upper()
df_dimcustomer

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,11000,26,AW00011000,None,jon,V,YANG,0,1971-10-06,M,...,Professional,Profesional,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles
1,11001,37,AW00011001,None,eugene,L,HUANG,0,1976-05-10,S,...,Professional,Profesional,Cadre,0,1,2243 W St.,None,1 (11) 500 555-0110,2011-01-15,0-1 Miles
2,11002,31,AW00011002,None,ruben,None,TORRES,0,1971-02-09,M,...,Professional,Profesional,Cadre,1,1,5844 Linden Land,None,1 (11) 500 555-0184,2011-01-07,2-5 Miles
3,11003,11,AW00011003,None,christy,None,ZHU,0,1973-08-14,S,...,Professional,Profesional,Cadre,0,1,1825 Village Pl.,None,1 (11) 500 555-0162,2010-12-29,5-10 Miles
4,11004,19,AW00011004,None,elizabeth,None,JOHNSON,0,1979-08-05,S,...,Professional,Profesional,Cadre,1,4,7553 Harness Circle,None,1 (11) 500 555-0131,2011-01-23,1-2 Miles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,29479,209,AW00029479,None,tommy,L,TANG,0,1969-06-30,M,...,Clerical,Administrativo,Employé,1,0,"111, rue Maillard",None,1 (11) 500 555-0136,2012-09-04,0-1 Miles
18480,29480,248,AW00029480,None,nina,W,RAJI,0,1977-05-06,S,...,Clerical,Administrativo,Employé,1,0,9 Katherine Drive,None,1 (11) 500 555-0146,2013-07-17,0-1 Miles
18481,29481,120,AW00029481,None,ivan,None,SURI,0,1965-07-04,S,...,Clerical,Administrativo,Employé,0,0,Knaackstr 4,None,1 (11) 500 555-0144,2011-08-13,0-1 Miles
18482,29482,179,AW00029482,None,clayton,None,ZHANG,0,1964-09-01,M,...,Clerical,Administrativo,Employé,1,0,"1080, quai de Grenelle",None,1 (11) 500 555-0137,2012-09-18,0-1 Miles


In [9]:
#Sulla colonna EmailAddress, utilizzando il metodo .str.split(), estraiamo nome utente e dominio
df_dimcustomer[["NomeUtente", "Dominio"]] = df_dimcustomer['EmailAddress'].str.split("@", expand=True) #inserisco il risultato dello split in due nuove colonne
df_dimcustomer

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance,NomeUtente,Dominio
0,11000,26,AW00011000,None,jon,V,YANG,0,1971-10-06,M,...,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles,jon24,adventure-works.com
1,11001,37,AW00011001,None,eugene,L,HUANG,0,1976-05-10,S,...,Cadre,0,1,2243 W St.,None,1 (11) 500 555-0110,2011-01-15,0-1 Miles,eugene10,adventure-works.com
2,11002,31,AW00011002,None,ruben,None,TORRES,0,1971-02-09,M,...,Cadre,1,1,5844 Linden Land,None,1 (11) 500 555-0184,2011-01-07,2-5 Miles,ruben35,adventure-works.com
3,11003,11,AW00011003,None,christy,None,ZHU,0,1973-08-14,S,...,Cadre,0,1,1825 Village Pl.,None,1 (11) 500 555-0162,2010-12-29,5-10 Miles,christy12,adventure-works.com
4,11004,19,AW00011004,None,elizabeth,None,JOHNSON,0,1979-08-05,S,...,Cadre,1,4,7553 Harness Circle,None,1 (11) 500 555-0131,2011-01-23,1-2 Miles,elizabeth5,adventure-works.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,29479,209,AW00029479,None,tommy,L,TANG,0,1969-06-30,M,...,Employé,1,0,"111, rue Maillard",None,1 (11) 500 555-0136,2012-09-04,0-1 Miles,tommy2,adventure-works.com
18480,29480,248,AW00029480,None,nina,W,RAJI,0,1977-05-06,S,...,Employé,1,0,9 Katherine Drive,None,1 (11) 500 555-0146,2013-07-17,0-1 Miles,nina21,adventure-works.com
18481,29481,120,AW00029481,None,ivan,None,SURI,0,1965-07-04,S,...,Employé,0,0,Knaackstr 4,None,1 (11) 500 555-0144,2011-08-13,0-1 Miles,ivan0,adventure-works.com
18482,29482,179,AW00029482,None,clayton,None,ZHANG,0,1964-09-01,M,...,Employé,1,0,"1080, quai de Grenelle",None,1 (11) 500 555-0137,2012-09-18,0-1 Miles,clayton0,adventure-works.com


In [10]:
#Sulla colonna Phone, estraiamo ogni parte del numero (ad es. da "1 (11) 500 555-0162" a ["1", "(11)", "500", "555-0162"])
df_dimcustomer[["Num1", "Num2", "Num3", "Num4"]] = df_dimcustomer['Phone'].str.split(" ", expand=True)#versione con split
df_dimcustomer

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,AddressLine2,Phone,DateFirstPurchase,CommuteDistance,NomeUtente,Dominio,Num1,Num2,Num3,Num4
0,11000,26,AW00011000,None,jon,V,YANG,0,1971-10-06,M,...,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles,jon24,adventure-works.com,1,(11),500,555-0162
1,11001,37,AW00011001,None,eugene,L,HUANG,0,1976-05-10,S,...,None,1 (11) 500 555-0110,2011-01-15,0-1 Miles,eugene10,adventure-works.com,1,(11),500,555-0110
2,11002,31,AW00011002,None,ruben,None,TORRES,0,1971-02-09,M,...,None,1 (11) 500 555-0184,2011-01-07,2-5 Miles,ruben35,adventure-works.com,1,(11),500,555-0184
3,11003,11,AW00011003,None,christy,None,ZHU,0,1973-08-14,S,...,None,1 (11) 500 555-0162,2010-12-29,5-10 Miles,christy12,adventure-works.com,1,(11),500,555-0162
4,11004,19,AW00011004,None,elizabeth,None,JOHNSON,0,1979-08-05,S,...,None,1 (11) 500 555-0131,2011-01-23,1-2 Miles,elizabeth5,adventure-works.com,1,(11),500,555-0131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,29479,209,AW00029479,None,tommy,L,TANG,0,1969-06-30,M,...,None,1 (11) 500 555-0136,2012-09-04,0-1 Miles,tommy2,adventure-works.com,1,(11),500,555-0136
18480,29480,248,AW00029480,None,nina,W,RAJI,0,1977-05-06,S,...,None,1 (11) 500 555-0146,2013-07-17,0-1 Miles,nina21,adventure-works.com,1,(11),500,555-0146
18481,29481,120,AW00029481,None,ivan,None,SURI,0,1965-07-04,S,...,None,1 (11) 500 555-0144,2011-08-13,0-1 Miles,ivan0,adventure-works.com,1,(11),500,555-0144
18482,29482,179,AW00029482,None,clayton,None,ZHANG,0,1964-09-01,M,...,None,1 (11) 500 555-0137,2012-09-18,0-1 Miles,clayton0,adventure-works.com,1,(11),500,555-0137


In [11]:
#versione con regex


In [12]:
#Utilizzando il metodo .str.contains(), estraiamo tutti gli indirizzi e-mail che contengono il numero "21"
df_dimcustomer["21_Numbers"] = df_dimcustomer['EmailAddress'].str.contains("21")
selected_emails = df_dimcustomer.loc[df_dimcustomer['21_Numbers'] == True, "EmailAddress"]
print("\nE-mail con il 21\n", selected_emails)


E-mail con il 21
 64          chase21@adventure-works.com
118         alvin21@adventure-works.com
130      caroline21@adventure-works.com
163       gabriel21@adventure-works.com
203          luis21@adventure-works.com
                      ...              
18337      hector21@adventure-works.com
18365      cheryl21@adventure-works.com
18435     stanley21@adventure-works.com
18440      rafael21@adventure-works.com
18480        nina21@adventure-works.com
Name: EmailAddress, Length: 431, dtype: object


• Estraiamo tutti gli indirizzi e-mail che contengono il numero "20" oppure il numero "10" • Calcolare la lunghezza di ogni indirizzo e-mail ed estrarre i cinque più lunghi e i cinque più corti • Modificare il dominio degli indirizzi e-mail da "adventure-works.com" a "aw-db.com" mediante il metodo .str.replace() • Dalla colonna AddressLine1 estraiamo tutti gli indirizzi che contengono la sottostringa "Street"

In [13]:
#Estraiamo tutti gli indirizzi e-mail che contengono il numero "20" oppure il numero "10"
df_dimcustomer["10_Numbers"] = df_dimcustomer['EmailAddress'].str.contains("10", na=False)
selected_emails_10 = df_dimcustomer.loc[df_dimcustomer['10_Numbers'], "EmailAddress"]

df_dimcustomer["20_Numbers"] = df_dimcustomer['EmailAddress'].str.contains("20", na=False)
selected_emails_20 = df_dimcustomer.loc[df_dimcustomer['20_Numbers'], "EmailAddress"]

print("\nE-mail con il 10\n", selected_emails_10)
print("\nE-mail con il 20\n", selected_emails_20)


E-mail con il 10
 1         eugene10@adventure-works.com
30       bethany10@adventure-works.com
32        denise10@adventure-works.com
44          adam10@adventure-works.com
100      latasha10@adventure-works.com
                     ...              
18300        joe10@adventure-works.com
18330      alexa10@adventure-works.com
18395      lydia10@adventure-works.com
18397     walter10@adventure-works.com
18418    lindsey10@adventure-works.com
Name: EmailAddress, Length: 559, dtype: object

E-mail con il 20
 10        jacquelyn20@adventure-works.com
22            ethan20@adventure-works.com
79           donald20@adventure-works.com
115           alvin20@adventure-works.com
146           karla20@adventure-works.com
                       ...               
18342        cassie20@adventure-works.com
18369        arturo20@adventure-works.com
18384       latasha20@adventure-works.com
18415         oscar20@adventure-works.com
18428    kristopher20@adventure-works.com
Name: EmailAddress, Leng

In [14]:
#Calcolare la lunghezza di ogni indirizzo e-mail ed estrarre i cinque più lunghi e i cinque più corti
df_dimcustomer["Email_Lenght"] = df_dimcustomer['EmailAddress'].str.len()
print(df_dimcustomer["Email_Lenght"].value_counts())
longest_emails = df_dimcustomer.nlargest(5,"Email_Lenght")[["EmailAddress","Email_Lenght"]] #per trovare le 5 più lunghe
print("\nLongest Emails\n", longest_emails)
shortest_emails = df_dimcustomer.nsmallest(5,"Email_Lenght")[["EmailAddress","Email_Lenght"]] #per trovare le 5 più corte
print("\nShortest Emails\n", shortest_emails)

Email_Lenght
28    4766
27    4762
29    2989
26    2886
30    1232
25     943
31     667
24     129
32      90
33      18
22       2
Name: count, dtype: int64

Longest Emails
                            EmailAddress  Email_Lenght
979   christopher25@adventure-works.com            33
1767  christopher18@adventure-works.com            33
2655  christopher23@adventure-works.com            33
3512  christopher27@adventure-works.com            33
4772  christopher10@adventure-works.com            33

Shortest Emails
                    EmailAddress  Email_Lenght
3974     k0@adventure-works.com            22
15299    y0@adventure-works.com            22
8      rob4@adventure-works.com            24
53     ana0@adventure-works.com            24
215    ana7@adventure-works.com            24


In [15]:
#Modificare il dominio degli indirizzi e-mail da "adventure-works.com" a "aw-db.com" mediante il metodo .str.replace()
df_dimcustomer['EmailAddress'] = df_dimcustomer['EmailAddress'].str.replace("adventure-works.com", "aw-db.com")
df_dimcustomer['EmailAddress']

,EmailAddress
0,jon24@aw-db.com
1,eugene10@aw-db.com
2,ruben35@aw-db.com
3,christy12@aw-db.com
4,elizabeth5@aw-db.com
...,...
18479,tommy2@aw-db.com
18480,nina21@aw-db.com
18481,ivan0@aw-db.com
18482,clayton0@aw-db.com


In [16]:
#Dalla colonna AddressLine1 estraiamo tutti gli indirizzi che contengono la sottostringa "Street"
street_address = df_dimcustomer['AddressLine1'].str.contains("Street")
street_address = df_dimcustomer.loc[street_address, "AddressLine1"]
street_address

,AddressLine1
7,942 Brook Street
12,4785 Scott Street
32,626 Bentley Street
63,4927 Virgil Street
125,8481 Zartop Street
...,...
18273,2155 Zartop Street
18292,3931 Scott Street
18323,134 Peachwillow Street
18333,1023 Hawkins Street


Dai beginner_datasets carichiamo in un DataFrame il file facebook.csv, che contiene dei post con data di pubblicazione, tipo (foto, video, …) e numero di reactions raccolte: • Con la funzione pd.to_datetime() convertiamo la colonna status_published in formato Timestamp • Utilizzando gli attributi .dt.year , .dt.month , .dt.day , .dt.dayofweek , .dt.dayofyear, ottieniamo informazioni specifiche sulle date delle transazioni, come l'anno, il mese, il giorno della settimana, il giorno dell'anno, eccetera

In [17]:
#carico il df
df_facebook = pd.read_csv("/content/drive/MyDrive/EPICODE/M4_PYTHON/datasets/datasets/beginner_datasets/facebook.csv")
df_facebook.info()
df_facebook.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7050 entries, 0 to 7049
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   status_id         7050 non-null   object
 1   status_type       7050 non-null   object
 2   status_published  7050 non-null   object
 3   num_reactions     7050 non-null   int64 
 4   num_comments      7050 non-null   int64 
 5   num_shares        7050 non-null   int64 
 6   num_likes         7050 non-null   int64 
 7   num_loves         7050 non-null   int64 
 8   num_wows          7050 non-null   int64 
 9   num_hahas         7050 non-null   int64 
 10  num_sads          7050 non-null   int64 
 11  num_angrys        7050 non-null   int64 
dtypes: int64(9), object(3)
memory usage: 661.1+ KB


,status_id,status_type,status_published,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys
0,246675545449582_1649696485147474,video,4/22/2018 6:00,529,512,262,432,92,3,1,1,0
1,246675545449582_1649426988507757,photo,4/21/2018 22:45,150,0,0,150,0,0,0,0,0
2,246675545449582_1648730588577397,video,4/21/2018 6:17,227,236,57,204,21,1,1,0,0
3,246675545449582_1648576705259452,photo,4/21/2018 2:29,111,0,0,111,0,0,0,0,0
4,246675545449582_1645700502213739,photo,4/18/2018 3:22,213,0,0,204,9,0,0,0,0


In [18]:
#Con la funzione pd.to_datetime() convertiamo la colonna status_published in formato Timestamp
df_facebook['status_published'] = pd.to_datetime(df_facebook['status_published'])
df_facebook['status_published']

,status_published
0,2018-04-22 06:00:00
1,2018-04-21 22:45:00
2,2018-04-21 06:17:00
3,2018-04-21 02:29:00
4,2018-04-18 03:22:00
...,...
7045,2016-09-24 02:58:00
7046,2016-09-23 11:19:00
7047,2016-09-21 23:03:00
7048,2016-09-20 00:43:00


In [19]:
#Utilizzando gli attributi .dt.year , .dt.month , .dt.day , .dt.dayofweek , .dt.dayofyear
# ottieniamo informazioni specifiche sulle date delle transazioni, come l'anno,
# il mese, il giorno della settimana, il giorno dell'anno
df_facebook["sp_year"] = df_facebook['status_published'].dt.year
df_facebook["sp_month"] = df_facebook['status_published'].dt.month
df_facebook["sp_day"] = df_facebook['status_published'].dt.day
df_facebook["sp_dayofweek"] = df_facebook['status_published'].dt.dayofweek
df_facebook["sp_dayofyear"] = df_facebook['status_published'].dt.dayofyear
df_facebook

,status_id,status_type,status_published,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys,sp_year,sp_month,sp_day,sp_dayofweek,sp_dayofyear
0,246675545449582_1649696485147474,video,2018-04-22 06:00:00,529,512,262,432,92,3,1,1,0,2018,4,22,6,112
1,246675545449582_1649426988507757,photo,2018-04-21 22:45:00,150,0,0,150,0,0,0,0,0,2018,4,21,5,111
2,246675545449582_1648730588577397,video,2018-04-21 06:17:00,227,236,57,204,21,1,1,0,0,2018,4,21,5,111
3,246675545449582_1648576705259452,photo,2018-04-21 02:29:00,111,0,0,111,0,0,0,0,0,2018,4,21,5,111
4,246675545449582_1645700502213739,photo,2018-04-18 03:22:00,213,0,0,204,9,0,0,0,0,2018,4,18,2,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7045,1050855161656896_1061863470556065,photo,2016-09-24 02:58:00,89,0,0,89,0,0,0,0,0,2016,9,24,5,268
7046,1050855161656896_1061334757275603,photo,2016-09-23 11:19:00,16,0,0,14,1,0,1,0,0,2016,9,23,4,267
7047,1050855161656896_1060126464063099,photo,2016-09-21 23:03:00,2,0,0,1,1,0,0,0,0,2016,9,21,2,265
7048,1050855161656896_1058663487542730,photo,2016-09-20 00:43:00,351,12,22,349,2,0,0,0,0,2016,9,20,1,264


In [20]:
#creo dei df per contare le vendite per anno, mese, giorno, ecc.
year = df_facebook['status_published'].dt.year.value_counts().sort_values(ascending=False)
month = df_facebook['status_published'].dt.month.value_counts().sort_values(ascending=False)
day = df_facebook['status_published'].dt.day.value_counts().sort_values(ascending=False)
dayofweek = df_facebook['status_published'].dt.dayofweek.value_counts().sort_values(ascending=False)
dayofyear = df_facebook['status_published'].dt.dayofyear.value_counts().sort_values(ascending=False)
print("\nStatus count by Year\n", year,"\n","\nStatus count by Month\n", month, "\n", "\nStatus count by Day\n", day, "\n", "\nStatus count by Day of Week\n", dayofweek, "\n", "\nStatus count by Day of Year\n", dayofyear)


Status count by Year
 status_published
2018    2550
2017    2253
2012     589
2016     588
2013     500
2015     337
2014     233
Name: count, dtype: int64 
 
Status count by Month
 status_published
12    823
5     733
11    674
3     666
4     635
6     604
1     585
2     513
8     499
9     486
10    485
7     347
Name: count, dtype: int64 
 
Status count by Day
 status_published
8     281
25    269
20    263
24    262
9     262
5     260
22    245
23    243
10    243
26    242
18    238
21    238
11    236
2     232
27    231
6     229
19    229
16    227
12    226
15    223
7     220
28    220
17    216
3     215
29    214
1     209
13    204
4     201
30    182
14    169
31    121
Name: count, dtype: int64 
 
Status count by Day of Week
 status_published
6    1059
1    1041
4    1010
0     994
3     983
5     982
2     981
Name: count, dtype: int64 
 
Status count by Day of Year
 status_published
158    57
160    47
162    46
159    45
145    42
       ..
195     4
182     3
170

Estraiamo solo i post relativi al 2012 • Estraiamo solo i post relativi a maggio 2018 • Confrontiamo il numero di post pubblicati nei weekend rispetto al numero di post pubblicati nel resto della settimana • Troviamo il primo e ultimo post pubblicati in ogni anno • Quanti tipi di post ci sono? E quanti per ogni tipo?

In [21]:
#post del 2012
df_facebook["filtro_2012"] = df_facebook["sp_year"] == 2012
post_2012 = df_facebook.loc[df_facebook['filtro_2012']]
post_2012

,status_id,status_type,status_published,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys,sp_year,sp_month,sp_day,sp_dayofweek,sp_dayofyear,filtro_2012
2046,246675545449582_303160673134402,photo,2012-12-23 03:11:00,0,0,0,0,0,0,0,0,0,2012,12,23,6,358,True
2047,246675545449582_302341609882975,photo,2012-12-21 06:02:00,4,1,0,4,0,0,0,0,0,2012,12,21,4,356,True
2048,246675545449582_299212316862571,photo,2012-12-13 11:36:00,3,0,0,3,0,0,0,0,0,2012,12,13,3,348,True
2049,246675545449582_290159194434550,photo,2012-11-18 09:50:00,5,1,1,5,0,0,0,0,0,2012,11,18,6,323,True
2050,246675545449582_290154177768385,photo,2012-11-18 09:16:00,2,0,0,2,0,0,0,0,0,2012,11,18,6,323,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2630,246675545449582_246688625448274,photo,2012-07-15 03:54:00,19,17,3,19,0,0,0,0,0,2012,7,15,6,197,True
2631,246675545449582_246686478781822,photo,2012-07-15 03:42:00,12,3,0,12,0,0,0,0,0,2012,7,15,6,197,True
2632,246675545449582_246684432115360,photo,2012-07-15 03:32:00,14,1,0,14,0,0,0,0,0,2012,7,15,6,197,True
2633,246675545449582_246678538782616,photo,2012-07-15 02:58:00,14,7,0,14,0,0,0,0,0,2012,7,15,6,197,True


In [22]:
#post del 2018
df_facebook["filtro_may_2018"] = (df_facebook["sp_year"] == 2018) & (df_facebook["sp_month"] == 5)
post_2018 = df_facebook.loc[df_facebook['filtro_may_2018']]
post_2018

,status_id,status_type,status_published,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys,sp_year,sp_month,sp_day,sp_dayofweek,sp_dayofyear,filtro_2012,filtro_may_2018
2690,134115277150304_264554297439734,photo,2018-05-31 12:17:00,11,0,0,10,0,1,0,0,0,2018,5,31,3,151,False,True
2691,134115277150304_264493694112461,photo,2018-05-31 08:37:00,56,4,2,55,1,0,0,0,0,2018,5,31,3,151,False,True
2692,134115277150304_264416247453539,photo,2018-05-31 03:12:00,72,13,1,69,3,0,0,0,0,2018,5,31,3,151,False,True
2693,134115277150304_264168277478336,photo,2018-05-30 08:46:00,10,0,0,10,0,0,0,0,0,2018,5,30,2,150,False,True
2694,134115277150304_264160757479088,photo,2018-05-30 08:24:00,56,3,12,54,2,0,0,0,0,2018,5,30,2,150,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,1050855161656896_1615666458509094,link,2018-05-04 01:27:00,12,7,7,12,0,0,0,0,0,2018,5,4,4,124,False,True
6340,1050855161656896_1614717551937318,photo,2018-05-03 01:15:00,63,21,1,62,1,0,0,0,0,2018,5,3,3,123,False,True
6341,1050855161656896_1614187798656960,photo,2018-05-02 12:17:00,218,105,153,212,2,4,0,0,0,2018,5,2,2,122,False,True
6342,1050855161656896_1613063202102753,video,2018-05-01 08:38:00,246,2373,475,108,130,5,2,1,0,2018,5,1,1,121,False,True


In [23]:
#Confrontiamo il numero di post pubblicati nei weekend rispetto al numero di post pubblicati nel resto della settimana
weekend_post = df_facebook[df_facebook['sp_dayofweek'].between(4,6)].value_counts("sp_dayofweek")
workday_post = df_facebook[df_facebook['sp_dayofweek'].between(0,3)].value_counts("sp_dayofweek")
weekend_post = weekend_post.rename({4:"Venerdì", 5:"Sabato", 6:"Domenica"})
workday_post = workday_post.rename({0:"Lunedì", 1:"Martedì", 2:"Mercoledì", 3:"Giovedì"})
print("\nPost pubblicati nel weekend\n", weekend_post,)
print("\nPost pubblicati nei giorni lavorativi\n", workday_post)


Post pubblicati nel weekend
 sp_dayofweek
Domenica    1059
Venerdì     1010
Sabato       982
Name: count, dtype: int64

Post pubblicati nei giorni lavorativi
 sp_dayofweek
Martedì      1041
Lunedì        994
Giovedì       983
Mercoledì     981
Name: count, dtype: int64


Dai beginner_dataset carichiamo in un DataFrame il file pokemon.csv:
• Tramite i metodi .isnull() e .sum() controlliamo se ci sono valori nulli nel dataset e contiamo quanti valori nulli ci sono in ogni colonna
• Ci sono valori nulli?
• Se sì, avrebbe senso cercare di riempirli? • Eliminiamo le righe che contengono valori nulli.


In [24]:
#importo il df
df_pokemon = pd.read_csv("/content/drive/MyDrive/EPICODE/M4_PYTHON/datasets/datasets/beginner_datasets/pokemon.csv")
df_pokemon.info() #vedo già da qui che la colonna "Type 2" potrebbe contenere valori nulli
df_pokemon.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [25]:
#Tramite i metodi .isnull() e .sum() controlliamo se ci sono valori nulli nel dataset e contiamo quanti valori nulli ci sono in ogni colonna

df_pokemon.isnull().sum() # "Type 2" contiene 386 valori nulli

#Sostituire i valori nulli non avrebbe senso perchè molti pokemon hanno un solo "Type".
#Essendo inoltre una variabile categorica non sono possibili sostituzioni con media o altre soluzioni utilizzate per variabili numeriche

,0
#,0
Name,0
Type 1,0
Type 2,386
Total,0
HP,0
Attack,0
Defense,0
Sp. Atk,0
Sp. Def,0


In [26]:
#eliminiamo le righe contenenti valori nulli
df_pokemon["NoType2"] = df_pokemon["Type 2"].isnull()
df_pokemon_nonull = df_pokemon[df_pokemon["NoType2"]==False]
print("\nValori nulli\n", df_pokemon_nonull.isnull().sum())
print()
df_pokemon_nonull.info();


Valori nulli
 #             0
Name          0
Type 1        0
Type 2        0
Total         0
HP            0
Attack        0
Defense       0
Sp. Atk       0
Sp. Def       0
Speed         0
Generation    0
Legendary     0
NoType2       0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Index: 414 entries, 0 to 799
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           414 non-null    int64 
 1   Name        414 non-null    object
 2   Type 1      414 non-null    object
 3   Type 2      414 non-null    object
 4   Total       414 non-null    int64 
 5   HP          414 non-null    int64 
 6   Attack      414 non-null    int64 
 7   Defense     414 non-null    int64 
 8   Sp. Atk     414 non-null    int64 
 9   Sp. Def     414 non-null    int64 
 10  Speed       414 non-null    int64 
 11  Generation  414 non-null    int64 
 12  Legendary   414 non-null    bool  
 13  NoType2     414 non-null    bool  
dtypes:

Dai beginner_dataset carichiamo in un DataFrame il file automobile.csv: • Ci sono valori nulli? Dove? Quanti? • Quali righe hanno un valore nullo nella colonna num-of-doors? • Esaminando i dati nel dataset, cerchiamo una logica per sostituire i valori nulli nella colonna num-of-doors

In [27]:
#importo il df
df_auto = pd.read_csv("/content/drive/MyDrive/EPICODE/M4_PYTHON/datasets/datasets/beginner_datasets/automobile.csv")
df_auto.info()
df_auto.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          202 non-null    int64  
 1   normalized-losses  165 non-null    float64
 2   make               202 non-null    object 
 3   fuel-type          202 non-null    object 
 4   aspiration         202 non-null    object 
 5   num-of-doors       200 non-null    object 
 6   body-style         202 non-null    object 
 7   drive-wheels       202 non-null    object 
 8   engine-location    202 non-null    object 
 9   wheel-base         202 non-null    float64
 10  length             202 non-null    float64
 11  width              202 non-null    float64
 12  height             202 non-null    float64
 13  curb-weight        202 non-null    int64  
 14  engine-type        202 non-null    object 
 15  num-of-cylinders   202 non-null    object 
 16  engine-size        202 non

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [28]:
# Ci sono valori nulli? Dove? Quanti? • Quali righe hanno un valore nullo nella colonna num-of-doors?
print(df_auto.isnull().sum())

#valori nulli presenti in normalized-losses e num-of-doors
#sostituisco i nulli di normalized-losses
df_auto["normlosses_null"] = df_auto["normalized-losses"].isnull()
df_auto['normalized-losses'].groupby(df_auto["symboling"]).mean() #media perdite normalizzate per classe di rischio
#una possibilità per gestire i valori nulli è quella di sostituirli col valore medio della classe di rischio corrispondente

symboling             0
normalized-losses    37
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  0
stroke                0
compression-ratio     0
horsepower            0
peak-rpm              0
city-mpg              0
highway-mpg           0
price                 0
dtype: int64


,normalized-losses
symboling,
-2,103.000000
-1,86.047619
0,113.166667
1,128.574468
2,125.689655
3,168.647059


In [29]:
#sostituisco i valori nulli con la media per classe di rischio corrispondente
df_auto["normalized-losses"] = df_auto["normalized-losses"].fillna(df_auto.groupby("symboling")["normalized-losses"].transform("mean"))
print(df_auto.isnull().sum())
df_auto

symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         2
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
normlosses_null      0
dtype: int64


,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,normlosses_null
0,3,168.647059,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495,True
1,3,168.647059,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500,True
2,1,128.574468,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500,True
3,2,164.000000,audi,gas,std,four,sedan,fwd,front,99.8,...,mpfi,3.19,3.4,10.0,102,5500,24,30,13950,False
4,2,164.000000,audi,gas,std,four,sedan,4wd,front,99.4,...,mpfi,3.19,3.4,8.0,115,5500,18,22,17450,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,-1,95.000000,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,mpfi,3.78,3.15,8.7,160,5300,19,25,19045,False
198,-1,95.000000,volvo,gas,std,four,sedan,rwd,front,109.1,...,mpfi,3.58,2.87,8.8,134,5500,18,23,21485,False
199,-1,95.000000,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,idi,3.01,3.4,23.0,106,4800,26,27,22470,False
200,-1,95.000000,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,mpfi,3.78,3.15,9.5,114,5400,19,25,22625,False


In [30]:
#conto i valori nulli in num_of_doors
df_auto['num-of-doors'].isnull().sum() #
filtro_nod = df_auto['num-of-doors'].isnull()
null_doors_auto = df_auto.loc[filtro_nod] #a queste due auto (dodge e mazda) manca il numero di porte
null_doors_auto

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,normlosses_null
26,1,148.000000,dodge,gas,turbo,NaN,sedan,fwd,front,93.7,...,mpfi,3.03,3.39,7.6,102,5500,24,30,8558,False
60,0,113.166667,mazda,diesel,std,NaN,sedan,fwd,front,98.8,...,idi,3.39,3.39,22.7,64,4650,36,42,10795,True


In [31]:
dodge = df_auto["make"] == "dodge" #filtro le dodge
df_auto.loc[dodge]#la dodge col valore nullo è una berlina e tutte le berline del marchio hanno 4 porte

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,normlosses_null
20,1,118.0,dodge,gas,std,two,hatchback,fwd,front,93.7,...,2bbl,2.97,3.23,9.41,68,5500,37,41,5572,False
21,1,118.0,dodge,gas,std,two,hatchback,fwd,front,93.7,...,2bbl,2.97,3.23,9.40,68,5500,31,38,6377,False
22,1,118.0,dodge,gas,turbo,two,hatchback,fwd,front,93.7,...,mpfi,3.03,3.39,7.60,102,5500,24,30,7957,False
23,1,148.0,dodge,gas,std,four,hatchback,fwd,front,93.7,...,2bbl,2.97,3.23,9.40,68,5500,31,38,6229,False
24,1,148.0,dodge,gas,std,four,sedan,fwd,front,93.7,...,2bbl,2.97,3.23,9.40,68,5500,31,38,6692,False
25,1,148.0,dodge,gas,std,four,sedan,fwd,front,93.7,...,2bbl,2.97,3.23,9.40,68,5500,31,38,7609,False
26,1,148.0,dodge,gas,turbo,NaN,sedan,fwd,front,93.7,...,mpfi,3.03,3.39,7.60,102,5500,24,30,8558,False
27,-1,110.0,dodge,gas,std,four,wagon,fwd,front,103.3,...,2bbl,3.34,3.46,8.50,88,5000,24,30,8921,False
28,3,145.0,dodge,gas,turbo,two,hatchback,fwd,front,95.9,...,mfi,3.6,3.9,7.00,145,5000,19,24,12964,False


In [32]:
mazda = df_auto["make"] == "mazda" #filtro le mazda
df_auto.loc[mazda] #anche le berline mazda hanno 4 porte

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,normlosses_null
47,1,104.000000,mazda,gas,std,two,hatchback,fwd,front,93.1,...,2bbl,3.03,3.15,9.0,68,5000,30,31,5195,False
48,1,104.000000,mazda,gas,std,two,hatchback,fwd,front,93.1,...,2bbl,3.03,3.15,9.0,68,5000,31,38,6095,False
49,1,104.000000,mazda,gas,std,two,hatchback,fwd,front,93.1,...,2bbl,3.03,3.15,9.0,68,5000,31,38,6795,False
50,1,113.000000,mazda,gas,std,four,sedan,fwd,front,93.1,...,2bbl,3.03,3.15,9.0,68,5000,31,38,6695,False
51,1,113.000000,mazda,gas,std,four,sedan,fwd,front,93.1,...,2bbl,3.08,3.15,9.0,68,5000,31,38,7395,False
52,3,150.000000,mazda,gas,std,two,hatchback,rwd,front,95.3,...,4bbl,?,?,9.4,101,6000,17,23,10945,False
53,3,150.000000,mazda,gas,std,two,hatchback,rwd,front,95.3,...,4bbl,?,?,9.4,101,6000,17,23,11845,False
54,3,150.000000,mazda,gas,std,two,hatchback,rwd,front,95.3,...,4bbl,?,?,9.4,101,6000,17,23,13645,False
55,3,150.000000,mazda,gas,std,two,hatchback,rwd,front,95.3,...,mpfi,?,?,9.4,135,6000,16,23,15645,False
56,1,129.000000,mazda,gas,std,two,hatchback,fwd,front,98.8,...,2bbl,3.39,3.39,8.6,84,4800,26,32,8845,False


In [33]:
#sostituisco NaN con four perchè entrambe berlina
df_auto['num-of-doors'] = df_auto["num-of-doors"].fillna("four")
df_auto.isnull().sum()

,0
symboling,0
normalized-losses,0
make,0
fuel-type,0
aspiration,0
num-of-doors,0
body-style,0
drive-wheels,0
engine-location,0
wheel-base,0


Abbiamo il seguente DataFrame che raccoglie le misurazioni di un sensore che misura la temperatura atmosferica giornaliera: import numpy as np, pandas as pd temp = pd.DataFrame({"Giorno": [0, 1, 2, 3, 4, 5, 6 7, 8, 9, 10, 11, 12], "Temperature": [18, 19, 18, np.nan, 21, 20, 20, np.nan, 21, 23, np.nan, 23, 24]}) • Il sensore a volte non funziona, dunque alcuni dati sono mancanti: quale sarebbe la migliore strategia per gestirli?

In [34]:
pd_temp = pd.DataFrame(

    {"Giorno": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
     "Temperature": [18, 19, 18, np.nan, 21, 20, 20, np.nan, 21, 23, np.nan, 23, 24]}

    )
pd_temp_adj = pd_temp["Temperature"].interpolate(method="linear")
pd_temp_adj



,Temperature
0,18.0
1,19.0
2,18.0
3,19.5
4,21.0
5,20.0
6,20.0
7,20.5
8,21.0
9,23.0


Nel pacchetto os della standard library c'è la funzione os.listdir() che permette di avere la lista dei nomi di file all'interno di una directory; senza input di default li cerca nella directory di lavoro corrente, altrimenti si può passare un path per esaminare una directory specifica, ad esempio os.listdir("mio_progetto/beginner_datasets/")
*   Nella directory dei beginner_datasets, quali sono i dataset che contengono dati nulli?
*  Dovremo usare un ciclo for per esaminare tutti i nomi dei file Dovremo selezionare solo i nomi di file con estensione .csv (quindi usare un costrutto if)
*   Nel corpo dovremo leggere di volta in volta il file in esame, e caricarlo in un DataFrame con la funzione .read_csv()
*  Sul DataFrame dovremo utilizzare il metodo .isna() per trovare la maschera booleana dei dati nulli Dovremo contare i dati nulli, utilizzando .sum(); potremmo doverlo utilizzare più di una volta
* Dovremo stampare, o memorizzare in una list, solo i nomi dei file che contengono dati nulli
















Con il supporto di gpt4o è stato creato uno script per l'automazione del riconoscimento e conteggio di dati nulli.

In [37]:
#SCRIPT PER AUTOMAZIONE DI RICONOSCIMENTO E CONTEGGIO DATI NULLI, CORREZIONE AUTOMATICA E CREAZIONE DI CSV CON DATI NULLI RIMOSSI

import os
import pandas as pd
import numpy as np
import json
from datetime import datetime

# === CONFIGURAZIONE ===
# Definizione delle directory di input, output e log
INPUT_DIR = "/content/drive/MyDrive/EPICODE/M4_PYTHON/datasets/datasets/beginner_datasets"
OUTPUT_DIR = "/content/drive/MyDrive/EPICODE/M4_PYTHON/datasets/datasets/output_cleaned"
LOG_DIR = "/content/drive/MyDrive/EPICODE/M4_PYTHON/datasets/datasets/log_dir"



# Creazione delle directory di output e log se non esistono
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# === FUNZIONI ===

def trova_file_con_nulli(directory):
    """
    Scansiona tutti i file .csv in una directory e restituisce
    un report dei file che contengono valori nulli.
    """
    report = []
    for nome_file in os.listdir(directory):
        if nome_file.endswith(".csv"):
            file_path = os.path.join(directory, nome_file)
            try:
                df = pd.read_csv(file_path)
                nulli_per_col = df.isna().sum()
                tot_null = nulli_per_col.sum()
                if tot_null > 0:
                    report.append({
                        "file": nome_file,
                        "tot_null": int(tot_null),
                        "colonne": nulli_per_col[nulli_per_col > 0].to_dict()
                    })
            except Exception as e:
                print(f"Errore nel leggere {nome_file}: {e}")
    return report

def applica_imputazione(df, strategia_log):
    """
    Applica una strategia automatica per riempire i valori nulli in un DataFrame.
    Registra la strategia utilizzata per ciascuna colonna, specificando il metodo e il valore usato.
    """
    for col in df.columns:
        if df[col].isnull().any():
            if pd.api.types.is_numeric_dtype(df[col]):
                skew = df[col].skew()
                metodo = "median" if abs(skew) > 1 else "mean"
                valore = df[col].median() if metodo == "median" else df[col].mean()
                df[col] = df[col].fillna(valore)

            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                metodo = "interpolate-time"
                valore = None
                df[col] = df[col].interpolate(method='time')

            else:
                mode = df[col].mode().dropna()
                metodo = "mode" if not mode.empty else "constant-missing"
                valore = mode[0] if not mode.empty else "missing"
                df[col] = df[col].fillna(valore)

            strategia_log.append({
                "colonna": col,
                "metodo": metodo,
                "valore_utilizzato": valore
            })

    return df

def salva_log(file_name, strategia_log, tot_nulli):
    """
    Salva un file di log JSON contenente le strategie applicate
    per ciascun file corretto.
    """
    log_data = {
        "file": file_name,
        "timestamp": datetime.now().isoformat(),
        "nulli_trovati": tot_nulli,
        "strategie_applicate": strategia_log
    }
    log_path = os.path.join(LOG_DIR, file_name.replace(".csv", "_log.json"))
    with open(log_path, "w") as f:
        json.dump(log_data, f, indent=2)

def processa_file(file_entry):
    """
    Carica il file, applica la pulizia automatica dei dati nulli,
    salva il file pulito e genera un log delle operazioni.
    """
    nome_file = file_entry['file']
    path_input = os.path.join(INPUT_DIR, nome_file)
    path_output = os.path.join(OUTPUT_DIR, f"cleaned_{nome_file}")
    df = pd.read_csv(path_input)
    strategia_log = []
    df_clean = applica_imputazione(df, strategia_log)
    df_clean.to_csv(path_output, index=False)
    salva_log(nome_file, strategia_log, file_entry['tot_null'])
    print(f"✅ {nome_file} corretto e salvato in '{path_output}'")

# === AVVIO DEL PROCESSO ===
print("\n🔍 Scansione dei file con dati nulli...")
report = trova_file_con_nulli(INPUT_DIR)

if not report:
    print("✅ Nessun file con valori nulli trovato.")
else:
    print("\n📋 File con dati nulli trovati:")
    for idx, r in enumerate(report):
        print(f"[{idx}] {r['file']} - {r['tot_null']} valori nulli")
        for col, count in r['colonne'].items():
            print(f"    ➤ {col}: {count}")

    for file_entry in report:
        processa_file(file_entry)

    print("\n📂 Tutti i file sono stati corretti e salvati con log dettagliati.")

    # === FASE INTERATTIVA FINALE ===
    scelta = input("\n🔎 Vuoi aprire uno dei file corretti per verificarlo manualmente? (s/n): ").strip().lower()
    if scelta == 's':
        cleaned_files = [f"cleaned_{entry['file']}" for entry in report]
        print("\n📁 File disponibili:")
        for idx, fname in enumerate(cleaned_files):
            print(f"[{idx}] {fname}")
        try:
            idx_scelto = int(input("\nInserisci l'indice del file che vuoi aprire: "))
            if 0 <= idx_scelto < len(cleaned_files):
                file_path = os.path.join(OUTPUT_DIR, cleaned_files[idx_scelto])
                print(f"\n📍 Il file è disponibile al percorso: {file_path}")
                print("📂 Puoi ora aprirlo e analizzarlo direttamente con strumenti esterni (Excel, Python, ecc.).")
            else:
                print("❌ Indice non valido.")
        except ValueError:
            print("❌ Inserimento non valido.")



🔍 Scansione dei file con dati nulli...

📋 File con dati nulli trovati:
[0] automobile.csv - 39 valori nulli
    ➤ normalized-losses: 37
    ➤ num-of-doors: 2
[1] france.csv - 66 valori nulli
    ➤ CustomerID: 66
[2] elections.csv - 52 valori nulli
    ➤ state_abbreviation: 52
[3] hepatitis.csv - 153 valori nulli
    ➤ STEROID: 1
    ➤ LIVER BIG: 9
    ➤ LIVER FIRM: 10
    ➤ SPLEEN PALPABLE: 4
    ➤ SPIDERS: 4
    ➤ ASCITES: 4
    ➤ VARICES: 4
    ➤ BILIRUBIN: 5
    ➤ ALK PHOSPHATE: 28
    ➤ SGOT: 3
    ➤ ALBUMIN: 15
    ➤ PROTIME: 66
[4] house.csv - 7829 valori nulli
    ➤ LotFrontage: 259
    ➤ Alley: 1369
    ➤ MasVnrType: 872
    ➤ MasVnrArea: 8
    ➤ BsmtQual: 37
    ➤ BsmtCond: 37
    ➤ BsmtExposure: 38
    ➤ BsmtFinType1: 37
    ➤ BsmtFinType2: 38
    ➤ Electrical: 1
    ➤ FireplaceQu: 690
    ➤ GarageType: 81
    ➤ GarageYrBlt: 81
    ➤ GarageFinish: 81
    ➤ GarageQual: 81
    ➤ GarageCond: 81
    ➤ PoolQC: 1453
    ➤ Fence: 1179
    ➤ MiscFeature: 1406
[5] income.csv - 4262 v

In [38]:
cleaned_auto = pd.read_csv("/content/drive/MyDrive/EPICODE/M4_PYTHON/datasets/datasets/output_cleaned/cleaned_automobile.csv")
cleaned_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          202 non-null    int64  
 1   normalized-losses  202 non-null    float64
 2   make               202 non-null    object 
 3   fuel-type          202 non-null    object 
 4   aspiration         202 non-null    object 
 5   num-of-doors       202 non-null    object 
 6   body-style         202 non-null    object 
 7   drive-wheels       202 non-null    object 
 8   engine-location    202 non-null    object 
 9   wheel-base         202 non-null    float64
 10  length             202 non-null    float64
 11  width              202 non-null    float64
 12  height             202 non-null    float64
 13  curb-weight        202 non-null    int64  
 14  engine-type        202 non-null    object 
 15  num-of-cylinders   202 non-null    object 
 16  engine-size        202 non